In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
# 定义参数
region = 'tianjin'
year = '2016'
startDate = year + '-01-01'
endDate = year + '-12-31'

In [ ]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# 移除无效像元
# .reproject('EPSG:3857',None,30)
# elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)


In [ ]:
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)  
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)
landsat_images = ee.ImageCollection(l8.merge(l7).merge(l5)).map(water_index)
Map.addLayer(l8.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
print(l5.size().getInfo( ))
print(l7.size().getInfo())
print(l8.size().getInfo())
print(landsat_images.size().getInfo())

# 筛选得出采样影像

In [ ]:
sample_image = landsat_images.map(maskSR)
print(sample_image.size().getInfo())

# 多水体频率数据集的类别范围确定

In [ ]:
#### 设置年份
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/MARYLAND/Maryland_nationwide_' + year
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

# 采样函数

## 只有永久和非水体

In [ ]:
# 设置采样个数
# number_land = 150
# number_water = 2000
number_land = 250
number_water = 1500

# 基于图像的样本采集函数
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'point_type':'permanent_water','Image_id':image.get('system:id')}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'point_type':'no_water','Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points]).flatten()
    return sample_points

## 季节性，随机选点

In [ ]:
## 设置采样个数
training = 1500

def season(image):
    season_points = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = season_points.map(lambda i : i.setMulti({'point_type':'season','Image_id':image.get('system:id')}))
    return water_points

# 基于影像，进行样本采集

In [ ]:
# 永久性水体 + 陆地
points_collection = sample_image.map(imageSample).flatten()
dataset_id = region + '_kmean100_noseason_v2' + year
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

In [ ]:
# 季节性水体
points_collection = sample_image.map(season).flatten()
dataset_id = region + '_kmean100_season_v2' + year
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

In [ ]:
folder = 'wuhan'
label = 'wuhan_new' + year + '_l78_awei0'
# label = 'tianjin_' + 'new2020_l78'
# csv格式
selector = ['Image_id','B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder=folder, file_format='csv', selectors=selector)

# shp格式
selector = ['Image_id','B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder=folder, file_format='shp', selectors=selector)